# Machine Learning Engineer Nanodegree

## Capstone Project - Smile Detector

In this project, CNNs are used to build models to detect if the person in the image is smiling or not.
CelibA dataset is used for this purpose - http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html
The data for this project is downloaded from Kaggle - https://www.kaggle.com/jessicali9530/celeba-dataset

In this notebook, a simple CNN model is developed from scratch. This is considered as Benchmark model based on which improved models will be worked upon.

In [1]:
# all imports
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
from keras.preprocessing import image                  
from tqdm import tqdm

Using TensorFlow backend.


In [5]:
IMG_H=218
IMG_W=178
IMG_D=3

In [6]:
# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    smile_files = np.array(data['filenames'])
    smile_targets = np_utils.to_categorical(np.array(data['target']), 2)
    return smile_files, smile_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('input/dataset/train')
valid_files, valid_targets = load_dataset('input/dataset/validate')
test_files, test_targets = load_dataset('input/dataset/test')

smile_names = [item[:-1] for item in sorted(glob("input/dataset/train/*/"))]

# print statistics about the dataset
print('There are %d total smile categories.' % len(smile_names))
print('There are %s total smile images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training smile images.' % len(train_files))
print('There are %d validation smile images.' % len(valid_files))
print('There are %d test smile images.'% len(test_files))

Using TensorFlow backend.


There are 2 total smile categories.
There are 15000 total smile images.

There are 10000 training smile images.
There are 2500 validation smile images.
There are 2500 test smile images.


In [8]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(IMG_H, IMG_W))
    # convert PIL.Image.Image type to 3D tensor with shape (IMG_H, IMG_W, IMG_D)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, IMG_H, IMG_W, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

---

## Creating a CNN to detect smile (from Scratch)

### Pre-process the Data

We rescale the images by dividing every pixel in every image by 255.

In [10]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 2500/2500 [00:34<00:00, 72.18it/s] 


### BenchModel Architecture
A simple CNN model is developed here considering alternating layers of Conv2D and MaxPooling. Dropout layers are also added to avoid overfitting. All Conv2D layers have relu activation. The final classification dense layer has softmax activation.


In [11]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(filters=16,kernel_size=2,padding='same',activation='relu',input_shape=(IMG_H,IMG_W,IMG_D)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
model.add(Conv2D(filters=128,kernel_size=2,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
model.add(GlobalAveragePooling2D())
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 218, 178, 16)      208       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 109, 89, 16)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 89, 32)       2080      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 54, 44, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 54, 44, 64)        8256      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 27, 22, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 27, 22, 64)       

### Compile the Model

In [12]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the Model

Training the model in the code cell below.  Using model checkpointing to save the model that attains the least validation loss.

In [13]:
from keras.callbacks import ModelCheckpoint  

EPOCHS = 20

checkpointer = ModelCheckpoint(filepath='input/saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=EPOCHS, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 10000 samples, validate on 2500 samples
Epoch 1/20


 3680/10000 [==========>...................] - ETA: 27:55 - loss: 0.7082 - accuracy: 0.400 - ETA: 17:19 - loss: 0.6722 - accuracy: 0.550 - ETA: 13:15 - loss: 0.6838 - accuracy: 0.550 - ETA: 11:32 - loss: 0.6845 - accuracy: 0.550 - ETA: 12:01 - loss: 0.6835 - accuracy: 0.560 - ETA: 11:04 - loss: 0.6965 - accuracy: 0.525 - ETA: 10:24 - loss: 0.6954 - accuracy: 0.535 - ETA: 9:46 - loss: 0.6935 - accuracy: 0.537 - ETA: 9:18 - loss: 0.7013 - accuracy: 0.50 - ETA: 8:53 - loss: 0.7013 - accuracy: 0.47 - ETA: 8:41 - loss: 0.7000 - accuracy: 0.48 - ETA: 8:42 - loss: 0.6982 - accuracy: 0.50 - ETA: 8:45 - loss: 0.6980 - accuracy: 0.50 - ETA: 8:48 - loss: 0.6972 - accuracy: 0.50 - ETA: 8:35 - loss: 0.6975 - accuracy: 0.50 - ETA: 8:33 - loss: 0.6978 - accuracy: 0.49 - ETA: 8:26 - loss: 0.6971 - accuracy: 0.50 - ETA: 8:18 - loss: 0.6957 - accuracy: 0.51 - ETA: 8:18 - loss: 0.6952 - accuracy: 0.51 - ETA: 8:12 - loss: 0.6932 - accuracy: 0.52 - ETA: 8:07 - loss: 0.6926 - accuracy: 0.52 - ETA: 8:03 - lo

 3720/10000 [==========>...................] - ETA: 5:13 - loss: 0.6981 - accuracy: 0.45 - ETA: 5:12 - loss: 0.6966 - accuracy: 0.47 - ETA: 5:05 - loss: 0.6917 - accuracy: 0.51 - ETA: 4:59 - loss: 0.6929 - accuracy: 0.51 - ETA: 4:55 - loss: 0.6945 - accuracy: 0.50 - ETA: 4:54 - loss: 0.6917 - accuracy: 0.50 - ETA: 5:01 - loss: 0.6923 - accuracy: 0.49 - ETA: 5:09 - loss: 0.6930 - accuracy: 0.48 - ETA: 5:06 - loss: 0.6939 - accuracy: 0.48 - ETA: 5:05 - loss: 0.6929 - accuracy: 0.50 - ETA: 5:04 - loss: 0.6905 - accuracy: 0.50 - ETA: 5:04 - loss: 0.6888 - accuracy: 0.50 - ETA: 5:03 - loss: 0.6855 - accuracy: 0.52 - ETA: 5:33 - loss: 0.6856 - accuracy: 0.52 - ETA: 5:29 - loss: 0.6871 - accuracy: 0.51 - ETA: 5:27 - loss: 0.6871 - accuracy: 0.51 - ETA: 5:24 - loss: 0.6879 - accuracy: 0.51 - ETA: 5:22 - loss: 0.6881 - accuracy: 0.51 - ETA: 5:21 - loss: 0.6894 - accuracy: 0.51 - ETA: 5:20 - loss: 0.6896 - accuracy: 0.51 - ETA: 5:25 - loss: 0.6904 - accuracy: 0.50 - ETA: 5:22 - loss: 0.6906 - ac

10000/10000 [==============================] - ETA: 2:08 - loss: 0.6884 - accuracy: 0.53 - ETA: 2:07 - loss: 0.6893 - accuracy: 0.53 - ETA: 2:06 - loss: 0.6892 - accuracy: 0.53 - ETA: 2:05 - loss: 0.6892 - accuracy: 0.53 - ETA: 2:04 - loss: 0.6892 - accuracy: 0.53 - ETA: 2:03 - loss: 0.6892 - accuracy: 0.53 - ETA: 2:02 - loss: 0.6892 - accuracy: 0.53 - ETA: 2:01 - loss: 0.6892 - accuracy: 0.53 - ETA: 2:00 - loss: 0.6893 - accuracy: 0.53 - ETA: 1:59 - loss: 0.6894 - accuracy: 0.53 - ETA: 1:59 - loss: 0.6894 - accuracy: 0.53 - ETA: 1:58 - loss: 0.6894 - accuracy: 0.53 - ETA: 1:57 - loss: 0.6893 - accuracy: 0.53 - ETA: 1:56 - loss: 0.6893 - accuracy: 0.53 - ETA: 1:55 - loss: 0.6894 - accuracy: 0.53 - ETA: 1:54 - loss: 0.6894 - accuracy: 0.53 - ETA: 1:53 - loss: 0.6894 - accuracy: 0.53 - ETA: 1:52 - loss: 0.6893 - accuracy: 0.53 - ETA: 1:51 - loss: 0.6893 - accuracy: 0.53 - ETA: 1:50 - loss: 0.6893 - accuracy: 0.53 - ETA: 1:49 - loss: 0.6894 - accuracy: 0.53 - ETA: 1:48 - loss: 0.6894 - ac

 3720/10000 [==========>...................] - ETA: 5:00 - loss: 0.7163 - accuracy: 0.50 - ETA: 5:11 - loss: 0.6965 - accuracy: 0.57 - ETA: 5:22 - loss: 0.6845 - accuracy: 0.60 - ETA: 5:22 - loss: 0.6815 - accuracy: 0.60 - ETA: 5:19 - loss: 0.6784 - accuracy: 0.59 - ETA: 5:12 - loss: 0.6841 - accuracy: 0.57 - ETA: 5:06 - loss: 0.6818 - accuracy: 0.58 - ETA: 5:04 - loss: 0.6757 - accuracy: 0.59 - ETA: 5:00 - loss: 0.6832 - accuracy: 0.56 - ETA: 4:58 - loss: 0.6779 - accuracy: 0.57 - ETA: 4:55 - loss: 0.6760 - accuracy: 0.57 - ETA: 4:53 - loss: 0.6728 - accuracy: 0.57 - ETA: 4:51 - loss: 0.6737 - accuracy: 0.57 - ETA: 4:50 - loss: 0.6722 - accuracy: 0.58 - ETA: 4:48 - loss: 0.6714 - accuracy: 0.58 - ETA: 4:47 - loss: 0.6791 - accuracy: 0.57 - ETA: 4:46 - loss: 0.6777 - accuracy: 0.58 - ETA: 4:44 - loss: 0.6761 - accuracy: 0.58 - ETA: 4:44 - loss: 0.6762 - accuracy: 0.58 - ETA: 4:43 - loss: 0.6763 - accuracy: 0.59 - ETA: 4:41 - loss: 0.6747 - accuracy: 0.59 - ETA: 4:40 - loss: 0.6729 - ac

...

 3720/10000 [==========>...................] - ETA: 4:17 - loss: 0.4409 - accuracy: 0.80 - ETA: 4:16 - loss: 0.4914 - accuracy: 0.77 - ETA: 4:27 - loss: 0.4661 - accuracy: 0.78 - ETA: 4:28 - loss: 0.5467 - accuracy: 0.75 - ETA: 4:27 - loss: 0.4994 - accuracy: 0.78 - ETA: 4:28 - loss: 0.5058 - accuracy: 0.77 - ETA: 4:28 - loss: 0.5045 - accuracy: 0.77 - ETA: 4:29 - loss: 0.4868 - accuracy: 0.78 - ETA: 4:28 - loss: 0.4810 - accuracy: 0.77 - ETA: 4:28 - loss: 0.4799 - accuracy: 0.78 - ETA: 4:27 - loss: 0.4857 - accuracy: 0.77 - ETA: 4:27 - loss: 0.4812 - accuracy: 0.78 - ETA: 4:27 - loss: 0.4723 - accuracy: 0.79 - ETA: 4:27 - loss: 0.4796 - accuracy: 0.78 - ETA: 4:26 - loss: 0.4684 - accuracy: 0.79 - ETA: 4:26 - loss: 0.4650 - accuracy: 0.80 - ETA: 4:25 - loss: 0.4675 - accuracy: 0.79 - ETA: 4:25 - loss: 0.4704 - accuracy: 0.79 - ETA: 4:24 - loss: 0.4601 - accuracy: 0.80 - ETA: 4:24 - loss: 0.4625 - accuracy: 0.79 - ETA: 4:23 - loss: 0.4774 - accuracy: 0.79 - ETA: 4:23 - loss: 0.4843 - ac

10000/10000 [==============================] - ETA: 1:12 - loss: 0.4779 - accuracy: 0.77 - ETA: 1:12 - loss: 0.4781 - accuracy: 0.77 - ETA: 1:11 - loss: 0.4775 - accuracy: 0.77 - ETA: 1:11 - loss: 0.4773 - accuracy: 0.77 - ETA: 1:10 - loss: 0.4767 - accuracy: 0.77 - ETA: 1:10 - loss: 0.4764 - accuracy: 0.77 - ETA: 1:09 - loss: 0.4765 - accuracy: 0.77 - ETA: 1:08 - loss: 0.4780 - accuracy: 0.77 - ETA: 1:08 - loss: 0.4780 - accuracy: 0.77 - ETA: 1:07 - loss: 0.4779 - accuracy: 0.77 - ETA: 1:07 - loss: 0.4776 - accuracy: 0.77 - ETA: 1:06 - loss: 0.4777 - accuracy: 0.77 - ETA: 1:05 - loss: 0.4778 - accuracy: 0.77 - ETA: 1:05 - loss: 0.4778 - accuracy: 0.77 - ETA: 1:04 - loss: 0.4779 - accuracy: 0.77 - ETA: 1:04 - loss: 0.4776 - accuracy: 0.77 - ETA: 1:03 - loss: 0.4777 - accuracy: 0.77 - ETA: 1:03 - loss: 0.4776 - accuracy: 0.77 - ETA: 1:02 - loss: 0.4775 - accuracy: 0.77 - ETA: 1:01 - loss: 0.4772 - accuracy: 0.77 - ETA: 1:01 - loss: 0.4774 - accuracy: 0.77 - ETA: 1:00 - loss: 0.4778 - ac

 3720/10000 [==========>...................] - ETA: 5:36 - loss: 0.4451 - accuracy: 0.80 - ETA: 5:36 - loss: 0.5255 - accuracy: 0.77 - ETA: 6:02 - loss: 0.4871 - accuracy: 0.81 - ETA: 6:00 - loss: 0.4590 - accuracy: 0.81 - ETA: 5:58 - loss: 0.4956 - accuracy: 0.78 - ETA: 5:52 - loss: 0.5025 - accuracy: 0.77 - ETA: 5:45 - loss: 0.4987 - accuracy: 0.76 - ETA: 5:43 - loss: 0.4820 - accuracy: 0.77 - ETA: 5:42 - loss: 0.5136 - accuracy: 0.75 - ETA: 5:40 - loss: 0.5111 - accuracy: 0.74 - ETA: 5:35 - loss: 0.5037 - accuracy: 0.75 - ETA: 5:32 - loss: 0.5104 - accuracy: 0.74 - ETA: 5:28 - loss: 0.5080 - accuracy: 0.74 - ETA: 5:30 - loss: 0.5052 - accuracy: 0.74 - ETA: 5:28 - loss: 0.5063 - accuracy: 0.74 - ETA: 5:26 - loss: 0.4959 - accuracy: 0.75 - ETA: 5:24 - loss: 0.4901 - accuracy: 0.75 - ETA: 5:20 - loss: 0.4876 - accuracy: 0.75 - ETA: 5:17 - loss: 0.4820 - accuracy: 0.75 - ETA: 5:14 - loss: 0.4822 - accuracy: 0.75 - ETA: 5:11 - loss: 0.4848 - accuracy: 0.75 - ETA: 5:10 - loss: 0.4882 - ac

 3720/10000 [==========>...................] - ETA: 4:21 - loss: 0.3189 - accuracy: 0.85 - ETA: 4:26 - loss: 0.3408 - accuracy: 0.87 - ETA: 4:28 - loss: 0.3841 - accuracy: 0.83 - ETA: 4:31 - loss: 0.4704 - accuracy: 0.78 - ETA: 4:30 - loss: 0.4407 - accuracy: 0.80 - ETA: 4:29 - loss: 0.4520 - accuracy: 0.80 - ETA: 4:30 - loss: 0.4294 - accuracy: 0.82 - ETA: 4:29 - loss: 0.4572 - accuracy: 0.79 - ETA: 4:29 - loss: 0.4471 - accuracy: 0.78 - ETA: 4:29 - loss: 0.4496 - accuracy: 0.79 - ETA: 4:31 - loss: 0.4415 - accuracy: 0.79 - ETA: 4:35 - loss: 0.4547 - accuracy: 0.80 - ETA: 4:37 - loss: 0.4719 - accuracy: 0.78 - ETA: 4:38 - loss: 0.4660 - accuracy: 0.78 - ETA: 4:37 - loss: 0.4608 - accuracy: 0.79 - ETA: 4:36 - loss: 0.4587 - accuracy: 0.78 - ETA: 4:35 - loss: 0.4609 - accuracy: 0.78 - ETA: 4:34 - loss: 0.4649 - accuracy: 0.78 - ETA: 4:33 - loss: 0.4717 - accuracy: 0.77 - ETA: 4:32 - loss: 0.4700 - accuracy: 0.78 - ETA: 4:31 - loss: 0.4697 - accuracy: 0.78 - ETA: 4:30 - loss: 0.4726 - ac

10000/10000 [==============================] - ETA: 1:15 - loss: 0.4457 - accuracy: 0.79 - ETA: 1:14 - loss: 0.4454 - accuracy: 0.79 - ETA: 1:13 - loss: 0.4459 - accuracy: 0.79 - ETA: 1:13 - loss: 0.4458 - accuracy: 0.79 - ETA: 1:12 - loss: 0.4464 - accuracy: 0.79 - ETA: 1:12 - loss: 0.4459 - accuracy: 0.79 - ETA: 1:11 - loss: 0.4455 - accuracy: 0.79 - ETA: 1:10 - loss: 0.4451 - accuracy: 0.79 - ETA: 1:10 - loss: 0.4449 - accuracy: 0.79 - ETA: 1:09 - loss: 0.4448 - accuracy: 0.79 - ETA: 1:09 - loss: 0.4449 - accuracy: 0.79 - ETA: 1:08 - loss: 0.4457 - accuracy: 0.79 - ETA: 1:07 - loss: 0.4452 - accuracy: 0.79 - ETA: 1:07 - loss: 0.4448 - accuracy: 0.79 - ETA: 1:06 - loss: 0.4447 - accuracy: 0.79 - ETA: 1:06 - loss: 0.4449 - accuracy: 0.79 - ETA: 1:05 - loss: 0.4446 - accuracy: 0.79 - ETA: 1:05 - loss: 0.4443 - accuracy: 0.79 - ETA: 1:04 - loss: 0.4438 - accuracy: 0.79 - ETA: 1:04 - loss: 0.4438 - accuracy: 0.79 - ETA: 1:03 - loss: 0.4441 - accuracy: 0.79 - ETA: 1:02 - loss: 0.4445 - ac

### Load the Model with the Best Validation Loss

In [14]:
model.load_weights('input/saved_models/weights.best.from_scratch.hdf5')

### Test the Model

Trying out the model on the test dataset images.

In [15]:
# get index of predicted smile detection for each image in test set
smile_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(smile_predictions)==np.argmax(test_targets, axis=1))/len(smile_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 81.0400%



Here, we see that a simple CNN model is developed and that its accuracy is 81.04%. This is considered as a Benchmark and we try to develop a model with a better accuracy considering augmentation, transfer learning, bottlenexk features, pretuning, etc in the other notebooks.